<a href="https://colab.research.google.com/github/Menon-Vineet/PPG-Codes/blob/main/Winner_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ STEP 1: Install & Import
!pip install -q qwen-vl-utils
from transformers import AutoProcessor, AutoModelForVision2Seq
from qwen_vl_utils import process_vision_info
from PIL import Image
import torch, re
import pandas as pd
from google.colab import files
import os, re, time, csv, random
import torch  # you already have transformers; this is for seeding
from transformers import AutoTokenizer, AutoModelForCausalLM

# ✅ STEP 2: Load Model
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForVision2Seq.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.float16).to(device)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 62.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2199: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [ ]:
# Run log (append-only CSV) and simple model metadata
RUN_LOG = "ppg_runs.csv"
MODEL_ID = "Qwen2.5-VL-7B-Instruct-AWQ"   # or getattr(model.config, "_name_or_path", "unknown")
MODEL_HASH = "unknown"                    # fill if you track it elsewhere

# Optional: set seeds for reproducibility of any stochastic ops
random.seed(0); torch.manual_seed(0)

def log_run(**kv):
    header = ["image_id","prompt","seed","model_id","model_hash",
              "aphids_qwen","area_qwen_mm2","density_cm2",
              "aphids_yolo","delta","thresholds","ts","accepted"]
    exists = os.path.exists(RUN_LOG)
    with open(RUN_LOG, "a", newline="") as f:
        w = csv.DictWriter(f, fieldnames=header)
        if not exists: w.writeheader()
        # fill missing fields with empty strings
        row = {k: kv.get(k, "") for k in header}
        w.writerow(row)


In [ ]:


# ✅ STEP 3: Upload Images
uploaded = files.upload()
records = []


Saving aphid.1.jpg to aphid.1.jpg
Saving aphid.2.jpg to aphid.2.jpg
Saving aphid.3.jpg to aphid.3.jpg
Saving aphid.4.jpg to aphid.4.jpg
Saving aphid.5.jpg to aphid.5.jpg
Saving aphid.6.jpg to aphid.6.jpg
Saving aphid.7.jpg to aphid.7.jpg
Saving aphid.8.jpg to aphid.8.jpg
Saving aphid.9.jpg to aphid.9.jpg
Saving aphid.10.jpg to aphid.10.jpg
Saving aphid.11.jpg to aphid.11.jpg
Saving aphid.12.jpg to aphid.12.jpg
Saving aphid.13.jpg to aphid.13.jpg
Saving aphid.14.jpg to aphid.14.jpg
Saving aphid.15.jpg to aphid.15.jpg
Saving aphid.16.jpg to aphid.16.jpg
Saving aphid.17.jpg to aphid.17.jpg
Saving aphid.18.jpg to aphid.18.jpg
Saving aphid.19.jpg to aphid.19.jpg
Saving aphid.20.jpg to aphid.20.jpg
Saving aphid.21.jpg to aphid.21.jpg
Saving aphid.22.jpg to aphid.22.jpg
Saving aphid.23.jpg to aphid.23.jpg
Saving aphid.24.jpg to aphid.24.jpg
Saving aphid.25.jpg to aphid.25.jpg
Saving aphid.26.jpg to aphid.26.jpg
Saving aphid.27.jpg to aphid.27.jpg
Saving aphid.28.jpg to aphid.28.jpg
Saving aph

In [ ]:


# ============================================
# ✅ STEP 4: Helper functions — robust parse, validation, density (per cm²)
# ============================================

# Grab only the assistant's final turn if the decoded text includes system/user/assistant preamble
def assistant_tail(text: str):
    marker = "\nassistant\n"
    return text.split(marker)[-1].strip()

# Accept the LAST occurrence of "total:<int>, area:<int>" anywhere in the text
PAIR_PATTERN = re.compile(r"total\s*[:=]\s*(\d+)\s*,\s*area\s*[:=]\s*(\d+)", re.IGNORECASE)

def parse_total_area(text: str):
    matches = PAIR_PATTERN.findall(text)
    if not matches:
        return None, None
    total, area_mm2 = matches[-1]
    return int(total), int(area_mm2)

def valid_ranges(aphids: int, area_mm2: int):
    return (
        aphids is not None and area_mm2 is not None and
        0 <= aphids <= 20000 and
        100 <= area_mm2 <= 2_000_000
    )

def density_per_cm2(aphids: int, area_mm2: int):
    # pests per cm² = aphids / (area_mm² / 100)
    return round(aphids / (area_mm2 / 100.0), 4)

# ============================================
# ✅ STEP 5: Inference Loop (guardrails + logging + cm² density)
# ============================================
for fname in uploaded:
    print(f"\n🔍 Processing {fname}")
    image = Image.open(fname).convert("RGB")

    # Prompt (tight format, no extra words)
    user_prompt = (
        "Look only at this leaf image. Return exactly two integers in this format: "
        "total:<aphid_count>, area:<leaf_area_mm2>. Do not add any extra words or units."
    )
    messages = [{
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": user_prompt}
        ]
    }]

    # Tokenize and process input
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(device)

    # Generate output
    outputs = model.generate(**inputs, max_new_tokens=64)
    output_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    print("📊 Raw Output:", output_text)

    # Robust parse from assistant tail
    tail = assistant_tail(output_text)
    aphids, area_mm2 = parse_total_area(tail)

    # Guardrail: reject if format invalid or ranges out of bounds
    if not valid_ranges(aphids, area_mm2):
        print("❌ Guardrail rejection: invalid format or out-of-range values")
        records.append({
            "Image": fname,
            "Total Aphids": "",
            "Leaf Area (mm²)": "",
            "Density (pests/cm²)": "",
            "Accepted": "No"
        })
        # log_run / MODEL_ID / MODEL_HASH assumed defined earlier
        log_run(image_id=fname, prompt=user_prompt, seed=0, model_id=MODEL_ID, model_hash=MODEL_HASH,
                aphids_qwen=aphids if aphids is not None else "", area_qwen_mm2=area_mm2 if area_mm2 is not None else "",
                density_cm2="", aphids_yolo="", delta="", thresholds="aphids<=20000;100<=area<=2e6",
                ts=time.time(), accepted="No")
        continue

    # Compute density per cm²
    dens = density_per_cm2(aphids, area_mm2)

    # Optional: δ cross-check (fill yolo_counts[fname] later when you add YOLO)
    delta = ""
    if 'yolo_counts' in globals() and isinstance(yolo_counts, dict) and fname in yolo_counts:
        yolo_c = yolo_counts[fname]
        delta_val = abs(aphids - yolo_c) / max(1, yolo_c)
        delta = round(delta_val, 4)
        if delta_val > 0.25:
            print(f"⚠️ Flag for review: δ={delta_val:.3f} > 0.25")

    # Append result
    records.append({
        "Image": fname,
        "Total Aphids": aphids,
        "Leaf Area (mm²)": area_mm2,
        "Density (pests/cm²)": dens,
        "Accepted": "Yes" if (delta == "" or float(delta) <= 0.25) else "Flag"
    })

    # Persist run log
    log_run(image_id=fname, prompt=user_prompt, seed=0, model_id=MODEL_ID, model_hash=MODEL_HASH,
            aphids_qwen=aphids, area_qwen_mm2=area_mm2, density_cm2=dens,
            aphids_yolo=(yolo_counts[fname] if ('yolo_counts' in globals() and fname in yolo_counts) else ""),
            delta=delta, thresholds="aphids<=20000;100<=area<=2e6;δ≤0.25", ts=time.time(),
            accepted="Yes" if (delta == "" or float(delta) <= 0.25) else "Flag")

# ============================================
# ✅ STEP 6: Export CSV
# ============================================
df = pd.DataFrame(records)
df = df.sort_values("Total Aphids", ascending=False, na_position="last").reset_index(drop=True)
df.to_csv("aphid_totals.csv", index=False)
df



🔍 Processing aphid.1.jpg
📊 Raw Output: system
You are a helpful assistant.
user
Look only at this leaf image. Return exactly two integers in this format: total:<aphid_count>, area:<leaf_area_mm2>. Do not add any extra words or units.
assistant
total:0, area:1500

🔍 Processing aphid.2.jpg
📊 Raw Output: system
You are a helpful assistant.
user
Look only at this leaf image. Return exactly two integers in this format: total:<aphid_count>, area:<leaf_area_mm2>. Do not add any extra words or units.
assistant
total:10, area:500

🔍 Processing aphid.3.jpg
📊 Raw Output: system
You are a helpful assistant.
user
Look only at this leaf image. Return exactly two integers in this format: total:<aphid_count>, area:<leaf_area_mm2>. Do not add any extra words or units.
assistant
total:1, area:300

🔍 Processing aphid.4.jpg
📊 Raw Output: system
You are a helpful assistant.
user
Look only at this leaf image. Return exactly two integers in this format: total:<aphid_count>, area:<leaf_area_mm2>. Do not add 

,Image,Total Aphids,Leaf Area (mm²),Density (pests/cm²),Accepted
0,aphid.2.jpg,10,500,2.0000,Yes
1,aphid.4.jpg,10,300,3.3333,Yes
2,aphid.31.jpg,10,300,3.3333,Yes
3,aphid.29.jpg,10,350,2.8571,Yes
4,aphid.30.jpg,10,350,2.8571,Yes
...,...,...,...,...,...
145,healthy.66.jpg,0,1500,0.0000,Yes
146,healthy.67.jpg,0,1500,0.0000,Yes
147,healthy.68.jpg,0,1500,0.0000,Yes
148,healthy.69.jpg,0,1500,0.0000,Yes


In [ ]:
# ✅ STEP 7: Export to CSV (timestamped filename YYYYMMDDHHMM)
from datetime import datetime

# Build timestamp (local time). For UTC, use: datetime.utcnow().strftime('%Y%m%d%H%M')
stamp = datetime.now().strftime('%Y%m%d%H%M')
out_csv = f"{stamp}.csv"

df = pd.DataFrame(records)
df = df.sort_values("Total Aphids", ascending=False, na_position="last").reset_index(drop=True)
df.to_csv(out_csv, index=False)
print(f"\n✅ Saved CSV → {out_csv}")

# Optional download for Colab
try:
    from google.colab import files
    files.download(out_csv)
except Exception as e:
    print(f"ℹ️ Download skipped: {e}")




✅ Saved CSV → 202508282009.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>